# 如何理解 Negative Log-Likelihood Loss（负对数似然损失）与 Cross-Entropy Loss（交叉熵损失）？

---

## 一、先从概率出发：模型输出是什么？

### 1.1 分类问题的目标

我们希望模型预测：

* 输入 $x$ 属于哪个类别 $y \in \{1, 2, ..., C\}$

理想的输出是：

$$
P(y = c \mid x)
$$

即：在给定输入 $x$ 的条件下，属于第 $c$ 类的概率。

➡️ 所以神经网络最后一层通常会输出一个**概率分布**，最常用的函数就是 Softmax。

---

### 1.2 Softmax 输出解释（用于多分类）

设模型输出一个向量 $\mathbf{z} = (z_1, z_2, ..., z_C)$（logits），
使用 Softmax 得到概率分布：

$$
\hat{y}_i = \frac{e^{z_i}}{\sum_{j=1}^C e^{z_j}} \quad \text{for } i = 1, ..., C
$$

其中 $\hat{y}_i$ 表示模型预测输入 $x$ 属于类别 $i$ 的概率。

---

## 二、Negative Log-Likelihood Loss（负对数似然损失）

### 2.1 似然函数（Likelihood Function）

从最大似然估计（MLE）出发，我们希望最大化：

$$
P(y = c \mid x; \theta)
$$

即模型在真实标签 $y = c$ 上的预测概率。

而在训练中，为了优化方便，我们**最小化负对数似然（Negative Log-Likelihood）**：

$$
\mathcal{L}_{\text{NLL}} = - \log P(y = c \mid x; \theta) = - \log \hat{y}_c
$$

* $\hat{y}_c$ 是 softmax 输出中对应真实标签 $y = c$ 的概率。
* 损失越小，说明模型越确信正确类别。

这就是负对数似然损失，简称 **NLL Loss**。

---

## 三、Cross-Entropy Loss（交叉熵损失）

### 3.1 定义

交叉熵（Cross-Entropy）来自信息论，用于衡量两个概率分布之间的差异：

$$
\text{CrossEntropy}(p, q) = - \sum_{i=1}^C p_i \log q_i
$$

其中：

* $p$ 是真实分布（通常是 one-hot 向量）
* $q$ 是模型预测的分布（通常是 softmax 输出）

在分类中，真实分布 $p$ 是：

$$
p_i =
\begin{cases}
1, & \text{if } i = c \ (\text{真实类别}) \\
0, & \text{otherwise}
\end{cases}
$$

所以：

$$
\mathcal{L}_{\text{CE}} = - \log q_c
$$

➡️ 这与 NLL 完全相同！

---

## 结论：Cross-Entropy Loss = Negative Log-Likelihood Loss（当真实标签为 One-hot）

| 中文术语       | 英文术语                               | 说明                             |
| ---------- | ---------------------------------- | ------------------------------ |
| 负对数似然损失    | Negative Log-Likelihood Loss (NLL) | 对正确类别的预测概率取对数再取负               |
| 交叉熵损失      | Cross-Entropy Loss                 | 信息论意义下的分布距离，one-hot 情况下等价于 NLL |
| softmax 输出 | Softmax Output                     | 多分类时的概率分布                      |
| one-hot 标签 | One-hot Label                      | 类别标签在 $p$ 中为1，其余为0             |

---

## 四、代码层面理解（PyTorch 示例）

```python
import torch
import torch.nn as nn

# 假设有3类，模型输出 logits
logits = torch.tensor([[2.0, 0.5, 1.2]])
labels = torch.tensor([0])  # 类别0是正确标签

# CrossEntropyLoss = log_softmax + NLLLoss
criterion = nn.CrossEntropyLoss()
loss = criterion(logits, labels)

print(loss)
```

 `CrossEntropyLoss` = `LogSoftmax + NLLLoss`

---

## 五、直觉解释：为什么是负对数？

* 概率越大，log 值越接近0（理想值）；
* 取负是为了使损失函数最小化时概率最大化；
* 损失值范围：\[0, +∞)，并具有良好的凸性（log 函数对正数是凸的）；
* 训练目标就是：**让正确类别的概率最大、损失最小**。

---

## 六、信息论视角：为什么叫“交叉熵”？

* 熵（Entropy）：自己对自己分布的不确定性

  $$
  H(p) = - \sum_i p_i \log p_i
  $$
* 交叉熵：使用另一个分布 $q$ 来编码真实分布 $p$ 的代价

  $$
  H(p, q) = - \sum_i p_i \log q_i
  $$

如果 $p$ 是 one-hot，交叉熵即为 $-\log q_c$，仍然是对正确类别预测概率的惩罚。

---

## 总结口诀

> “模型要会判对错，预测输出得用 soft；
> 正确类别 log 一取，负号加上损失落；
> CrossEntropy 和 NLL，一样意思两种说\~”

---

## 中英术语速查表

| 中文术语       | 英文术语                                | 解释                   |
| ---------- | ----------------------------------- | -------------------- |
| 对数似然       | Log-likelihood                      | 概率取对数后进行优化           |
| 负对数似然损失    | Negative Log-likelihood Loss        | 损失 = -log(预测正确类别的概率) |
| 交叉熵损失      | Cross-Entropy Loss                  | 测量真实分布和预测分布差异        |
| 熵          | Entropy                             | 测量分布的不确定性            |
| softmax 函数 | Softmax Function                    | 将模型输出转换为概率分布         |
| 最大似然估计     | Maximum Likelihood Estimation (MLE) | 找到使观测数据概率最大的参数       |

---



# 多层感知机（MLP）中如何计算梯度？如何进行反向传播？

---

## 一、什么是多层感知机（MLP）？

### 1.1 基本结构

多层感知机是一种**前馈神经网络（Feedforward Neural Network）**，由多个线性层（Linear）和非线性激活函数（Activation）堆叠而成。

以下是一个 2 层感知机（1 个隐藏层）：

```
Input x  →  Linear(W₁, b₁) → ReLU/σ → Linear(W₂, b₂) → softmax → Loss
```

---

## 二、前向传播（Forward Pass）

设输入为 $\boldsymbol{x} \in \mathbb{R}^d$，隐藏层神经元为 $h$，输出类别为 $C$。

### 2.1 第一层（输入 → 隐藏层）：

$$
\boldsymbol{z}^{[1]} = W^{[1]} \boldsymbol{x} + \boldsymbol{b}^{[1]} \quad \text{(线性变换)}
$$

$$
\boldsymbol{a}^{[1]} = \sigma(\boldsymbol{z}^{[1]}) \quad \text{(非线性激活，例如 ReLU, Sigmoid)}
$$

### 2.2 第二层（隐藏层 → 输出层）：

$$
\boldsymbol{z}^{[2]} = W^{[2]} \boldsymbol{a}^{[1]} + \boldsymbol{b}^{[2]}
$$

$$
\boldsymbol{\hat{y}} = \text{softmax}(\boldsymbol{z}^{[2]}) \quad \text{(输出概率)}
$$

---

## 三、定义损失函数（Loss Function）

设真实标签为 one-hot 向量 $\boldsymbol{y}$，常用的交叉熵损失为：

$$
\mathcal{L} = - \sum_{i=1}^C y_i \log \hat{y}_i
$$

---

## 四、反向传播（Backpropagation）

核心思想是：使用**链式法则（Chain Rule）**，从最后一层开始依次向前计算梯度。

---

### 4.1 步骤一：输出层 → 损失函数对 $\boldsymbol{z}^{[2]}$ 的梯度

$$
\frac{\partial \mathcal{L}}{\partial \boldsymbol{z}^{[2]}} = \boldsymbol{\hat{y}} - \boldsymbol{y}
$$

解释：

* softmax + cross-entropy 组合可简化梯度；
* 输出层的梯度是预测值与真实值的差。

---

### 4.2 步骤二：隐藏层 → 损失函数对 $W^{[2]}$、$b^{[2]}$ 的梯度

$$
\frac{\partial \mathcal{L}}{\partial W^{[2]}} = \frac{\partial \mathcal{L}}{\partial \boldsymbol{z}^{[2]}} \cdot (\boldsymbol{a}^{[1]})^T
$$

$$
\frac{\partial \mathcal{L}}{\partial \boldsymbol{b}^{[2]}} = \frac{\partial \mathcal{L}}{\partial \boldsymbol{z}^{[2]}}
$$

---

### 4.3 步骤三：中间层的链式法则展开

继续往前传播，注意中间有非线性激活函数。

#### 计算隐藏层激活前的梯度：

$$
\frac{\partial \mathcal{L}}{\partial \boldsymbol{a}^{[1]}} = (W^{[2]})^T \cdot \frac{\partial \mathcal{L}}{\partial \boldsymbol{z}^{[2]}}
$$

#### 计算隐藏层线性输入 $\boldsymbol{z}^{[1]}$ 的梯度：

$$
\frac{\partial \mathcal{L}}{\partial \boldsymbol{z}^{[1]}} = \frac{\partial \mathcal{L}}{\partial \boldsymbol{a}^{[1]}} \circ \sigma'(\boldsymbol{z}^{[1]})
$$

* 这里用的是 Hadamard product（元素乘），符号 $\circ$。
* $\sigma'$ 是激活函数的导数，如 ReLU 的导数是 0 或 1。

---

### 4.4 步骤四：输入层 → 权重 $W^{[1]}, b^{[1]}$ 的梯度

$$
\frac{\partial \mathcal{L}}{\partial W^{[1]}} = \frac{\partial \mathcal{L}}{\partial \boldsymbol{z}^{[1]}} \cdot \boldsymbol{x}^T
$$

$$
\frac{\partial \mathcal{L}}{\partial \boldsymbol{b}^{[1]}} = \frac{\partial \mathcal{L}}{\partial \boldsymbol{z}^{[1]}}
$$

---

## 总结：每一层反向传播的计算规律

| 名称    | 梯度公式                                                                    | 含义                  |
| ----- | ----------------------------------------------------------------------- | ------------------- |
| 输出层误差 | $\delta^{[2]} = \hat{y} - y$                                            | softmax + CE 简化后的结果 |
| 隐藏层误差 | $\delta^{[1]} = (W^{[2]})^T \delta^{[2]} \circ \sigma'(z^{[1]})$        | 链式法则传播，乘以激活函数导数     |
| 权重梯度  | $\frac{\partial \mathcal{L}}{\partial W} = \delta \cdot \text{上一层输出}^T$ |                     |
| 偏置梯度  | $\frac{\partial \mathcal{L}}{\partial b} = \delta$                      |                     |

---

## 用图示理解（建议画出）

```
Forward:
    x  →  z1 → a1 → z2 → a2=ŷ
            ↑       ↑
           W1,b1   W2,b2

Backward:
    ∂L/∂ŷ → ∂L/∂z2 → ∂L/∂a1 → ∂L/∂z1
           ↓        ↓        ↓
          ∂L/∂W2   ∂L/∂W1   ...
```

---

## 中英术语速查表

| 中文术语    | 英文术语                        | 定义                     |
| ------- | --------------------------- | ---------------------- |
| 多层感知机   | Multilayer Perceptron (MLP) | 一种前馈神经网络               |
| 前向传播    | Forward Propagation         | 从输入到输出                 |
| 反向传播    | Backward Propagation        | 计算损失对参数的梯度             |
| 链式法则    | Chain Rule                  | 求导时从外向内乘导数             |
| 激活函数    | Activation Function         | 加入非线性能力，如 ReLU、Sigmoid |
| 权重矩阵    | Weight Matrix               | 每层的线性变换参数              |
| 偏置项     | Bias Term                   | 每层加上的常数项               |
| 梯度      | Gradient                    | 损失函数对参数的偏导数            |
| Softmax | Softmax                     | 输出多分类概率分布函数            |

---

## ✅ 总结口诀

> “前向传播层层推，线性激活来相随；
> 损失函数算差异，链式反传不容怠；
> 误差乘导是精髓，参数更新靠它飞\~”

